# Example of Different Model Training 
The following script illustrates the multiple methods of prepairing the data and training models. 

In [ ]:
!python prep_wav.py -h

# Normalized Snapshot Training without Distinct Test Set
This is the simplest method available. All that is required is a recording of the clean input data and the processed/dirty output data at one condition (setting of knobs on the analog hardware).

The ```-s``` option denotes a "snapshot" model. The first argument is the clean input wav file, the second argument is the processed output wave file. Optionally, the third and fourth arguments may be a clean test input wav file and a processed test output wav file. An example is shown below.

The training will seed all random number generators with 39 and will run for 100 epochs.

### CAUTION: The test data set is the same as the validaiton set. This will give a biased result on the performance of the model. 

In [ ]:
!python prep_wav.py "LPB1_ex1"\
-s "./Recordings/20211003_LPB-1_5_min_Pink_Ramp_Clean.wav" \
"./Recordings/20211003_LPB-1_5_min_Pink_Ramp_Dirty.wav" \
 --normalize true

In [ ]:
!python dist_model_recnet.py -l RNN3-LPB1-ex1 -eps 5 --seed 39

# Snapshot with Random Training/Validation Split and Separate Test Set
The following settings perform a random 20% split on between the training and validation data. The split works by slicing the input audio into 100 even slices and chosing a certain number of them. Also here is defined a separate test data set which prevents the biasing of the results from the validation information leaking. 

### CAUTION: The random split between training and validation data may give unexpected results because the size of the devisions is long enough to maintain time dependency which also means that the Central Limit Theorm is not valid. 

In [ ]:
!python prep_wav.py "LPB1_ex2"\
-s "./Recordings/20211003_LPB-1_5_min_Pink_Ramp_Clean.wav" \
"./Recordings/20211003_LPB-1_5_min_Pink_Ramp_Dirty.wav" \
"./Recordings/20211003_LPB-1_Natalie_Guitar_Clean.wav" \
"./Recordings/20211003_LPB-1_Natalie_Guitar_Dirty.wav" \
-rs 20

In [ ]:
!python dist_model_recnet.py -l RNN3-LPB1-ex2 -eps 5


# Parameterized Model
The following settings prepair the data to create a parameterized model. A parameterization configuration file must be created. An example configuration file is included. The example is for one parameter, but any number of parameters up to 65355-(number of audio chanels) parameters may be defined. The limit is due to the number of channels allowed in a WAV file. 

The training instructions must then be modified to included the extra inputs to the model as ```-is <parameters + audio channels>``` That is, the number of inputs to the model must be the number of parameters plus the number of audio chanels. So for our example, ```-is 2```.

As is the default, the trainging and validation data is split on modulus of 5 which gives a 20 percent split.


In [ ]:
!python prep_wav.py "LPB1_Parameterized" -p "./Configs/Parameterization-Config.json"


In [ ]:
!python dist_model_recnet.py -l RNN3-LPB1_Parameterized -eps 5 --seed 39 -lm False -is 2